In [ ]:
# import h5py
# import pandas as pd
# import numpy as np

# # Step 1: Load the data matrix and metadata from the .gctx file
# gctx_file = "../data/raw/level3_beta_ctl_n188708x12328.gctx"
# h5_file = "../data/raw/compound_control_dataset_all.h5"

# chunk_size = 10000  # Number of rows to process at a time

# with h5py.File(gctx_file, "r") as f:
#     # Access data matrix and metadata
#     data = f["/0/DATA/0/matrix"]
#     row_ids = f["/0/META/COL/id"][:].astype(str)  # All sample IDs (rows)
#     col_ids = f["/0/META/ROW/id"][:].astype(int)  # All gene IDs (columns)

#     # Load metadata files
#     gene_metadata = pd.read_csv("../data/raw/geneinfo_beta.txt", sep="\t")
#     pert_info = pd.read_csv("../data/raw/instinfo_beta.txt", sep="\t")

#     # Filter perturbation metadata
#     filtered_pert_info = pert_info[
#         (pert_info["qc_pass"] == 1.0) & (pert_info["pert_type"] == "ctl_vehicle")
#     ].copy()
#     filtered_pert_info.set_index("sample_id", inplace=True)

#     # Filter valid sample IDs
#     valid_sample_ids = pd.Index(row_ids).intersection(filtered_pert_info.index)
#     filtered_pert_info = filtered_pert_info.loc[valid_sample_ids]
#     filtered_pert_info.sort_index(inplace=True)

#     # Create a mapping of sample IDs to row indices
#     row_idx_map = {rid: idx for idx, rid in enumerate(row_ids)}

#     # Compute row indices for valid samples
#     filtered_row_indices = [row_idx_map[sid] for sid in valid_sample_ids]

# # Step 2: Write the filtered data matrix and metadata in chunks
# with h5py.File(h5_file, "w") as h5f:
#     # Pre-allocate space for the dataset in the HDF5 file
#     data_shape = (len(filtered_row_indices), len(col_ids))
#     data_dset = h5f.create_dataset(
#         "data", shape=data_shape, dtype="float32", compression="gzip"
#     )

#     # Write row (sample) metadata
#     for column in filtered_pert_info.columns:
#         h5f.create_dataset(
#             f"row_metadata/{column}",
#             data=filtered_pert_info[column].values.astype("S"),
#         )

#     # Write column (gene) metadata
#     for column in gene_metadata.columns:
#         h5f.create_dataset(
#             f"col_metadata/{column}",
#             data=gene_metadata[column].values.astype("S"),
#         )

#     # Write row and column IDs
#     h5f.create_dataset("row_ids", data=valid_sample_ids.values.astype("S"))
#     gene_mapping = gene_metadata.set_index("gene_id")["gene_symbol"].to_dict()
#     filtered_col_symbols = [gene_mapping.get(gid, f"Gene_{gid}") for gid in col_ids]
#     h5f.create_dataset(
#         "col_ids", data=[x.encode("utf-8") for x in filtered_col_symbols]
#     )

#     # Process and write data in chunks
#     for i in range(0, len(filtered_row_indices), chunk_size):
#         chunk_indices = filtered_row_indices[i : i + chunk_size]  # Current chunk
#         chunk_data = data[chunk_indices, :]  # Load chunk from the data matrix
#         data_dset[i : i + len(chunk_indices), :] = chunk_data  # Write to dataset

# print(f"Filtered data and metadata successfully written to {h5_file}.")

In [1]:
# import h5py
# import pandas as pd

# # Step 1: Load the data matrix and metadata from the .gctx file
# gctx_file = "../data/raw/level3_beta_ctl_n188708x12328.gctx"

# with h5py.File(gctx_file, "r") as f:
#     # Access the entire data matrix and metadata
#     data = f["/0/DATA/0/matrix"]
#     row_ids = f["/0/META/COL/id"][:].astype(str)  # Sample IDs (rows)
#     col_ids = f["/0/META/ROW/id"][:].astype(int)  # Gene IDs (columns)

#     # Load metadata files
#     gene_metadata = pd.read_csv("../data/raw/geneinfo_beta.txt", sep="\t")
#     pert_info = pd.read_csv("../data/raw/instinfo_beta.txt", sep="\t")

#     # Filter perturbation metadata
#     filtered_pert_info = pert_info[
#         (pert_info["qc_pass"] == 1.0) & (pert_info["pert_type"] == "ctl_vehicle")
#     ].copy()
#     filtered_pert_info.set_index("sample_id", inplace=True)

#     # STEP A: Reduce filtered_pert_info to match row_ids
#     subset_row_ids = pd.Index(row_ids)
#     subset_row_ids = subset_row_ids.intersection(filtered_pert_info.index)
#     subset_pert_info = filtered_pert_info.loc[subset_row_ids].copy()
#     subset_pert_info.sort_index(inplace=True)

#     # Keep a separate column for sample_id (while retaining it as index)
#     subset_pert_info["sample_id"] = subset_pert_info.index

#     # STEP B: Recompute the row indices in the data matrix for valid sample_ids
#     row_idx_map = {rid: idx for idx, rid in enumerate(row_ids)}
#     filtered_row_indices = [row_idx_map[sid] for sid in subset_pert_info.index]

#     # Filter the data matrix to match the final subset of row indices
#     filtered_data_matrix = data[filtered_row_indices, :]

# # Step 2: Map gene IDs to gene symbols
# gene_mapping = gene_metadata.set_index("gene_id")["gene_symbol"].to_dict()
# filtered_col_symbols = [gene_mapping.get(gid, f"Gene_{gid}") for gid in col_ids]

# # Step 3: Write the filtered data and metadata to an HDF5 file
# h5_file = "../data/raw/compound_control_dataset.h5"

# with h5py.File(h5_file, "w") as h5f:
#     # Write the filtered data matrix
#     h5f.create_dataset("data", data=filtered_data_matrix, compression="gzip")

#     # Write row (sample) metadata
#     for column in subset_pert_info.columns:
#         h5f.create_dataset(
#             f"row_metadata/{column}",
#             data=subset_pert_info[column].values.astype("S"),
#         )

#     # Write column (gene) metadata
#     for column in gene_metadata.columns:
#         h5f.create_dataset(
#             f"col_metadata/{column}",
#             data=gene_metadata[column].values.astype("S"),
#         )

#     # Write final row and column IDs
#     h5f.create_dataset("row_ids", data=subset_pert_info.index.values.astype("S"))
#     h5f.create_dataset(
#         "col_ids", data=[x.encode("utf-8") for x in filtered_col_symbols]
#     )

# print(f"Filtered data and metadata successfully written to {h5_file}.")

C:\Users\20191678\AppData\Local\Temp/ipykernel_23136/3695529972.py:15: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  pert_info = pd.read_csv("../data/raw/instinfo_beta.txt", sep="\t")


Filtered data and metadata successfully written to ../data/raw/compound_control_dataset.h5.


In [7]:
# h5_file = "../data/raw/compound_control_dataset.h5"
# with h5py.File(h5_file, "r") as h5f:
#     # Load the data matrix
#     data_matrix = h5f["data"][:]

#     # Decode row metadata
#     row_metadata = {
#         key: h5f[f"row_metadata/{key}"][:].astype(str) for key in h5f["row_metadata"]
#     }

#     # Decode column metadata
#     col_metadata = {
#         key: h5f[f"col_metadata/{key}"][:].astype(str) for key in h5f["col_metadata"]
#     }

#     # Load and decode row and column IDs
#     row_ids = h5f["row_ids"][:].astype(str)
#     col_ids = h5f["col_ids"][:].astype(str)

# # Convert metadata to DataFrames for ease of use (optional)
# import pandas as pd

# row_metadata_df = pd.DataFrame(row_metadata)
# col_metadata_df = pd.DataFrame(col_metadata)


In [ ]:
import h5py
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict





# ------------------------------
# Example usage with DataLoader
# ------------------------------
if __name__ == "__main__":
    from torch.utils.data import DataLoader

    controls = "../data/raw/compound_control_dataset.h5"
    perturbations = "../data/raw/compound_pertubation_dataset.h5"

    # Create dataset with default 'normalize=False'
    dataset = GeneExpressionDataset(
        controls_file=controls,
        perturbations_file=perturbations,
        plate_column="det_plate",
        normalize=False,
        n_rows=1000,  # Debug limit
    )

    print(f"Dataset has {len(dataset)} (control, pert) pairs.")

    # DataLoader
    data_loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=0)

    # Example iteration
    for batch in data_loader:
        features = batch["features"]
        labels = batch["labels"]
        meta = batch["metadata"]
        print("Features Shape:", features.shape)  # (batch_size, num_genes)
        print("Labels Shape:", labels.shape)  # (batch_size, num_genes)
        print("Metadata sample:", meta["control_metadata"], meta["pert_metadata"])
        break

Dataset has 23389 (control, pert) pairs.
Features Shape: torch.Size([32, 12328])
Labels Shape: torch.Size([32, 12328])
Metadata sample: {'bead_batch': ['b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15', 'b15'], 'build_name': ['nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan', 'nan'], 'cell_iname': ['NCIH2073', 'A375', 'HEPG2', 'A375', 'HT29', 'HEPG2', 'HEPG2', 'A375', 'NCIH2073', 'NCIH1975', 'A375', 'NCIH2073', 'A375', 'A549', 'HT29', 'H1975', 'A549', 'A375', 'NCIH2073', 'NCIH2073', 'A549', 'HEPG2', 'A549', 'A375', 'NCIH1975', 'HEPG2', 'H1975', 'HEPG2', 'HEPG2', 'HEPG2', 'HT29', 'NCIH508'], 'cell_mfc_name': ['NCIH2073', 'A375', 'HEPG2', 'A375', 'HT29', 'HEPG2'

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Suppose you already have the GeneExpressionDataset defined as in the previous discussion:
# from your_file import GeneExpressionDataset


class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim=512, output_dim=None):
        """
        A simple MLP that predicts perturbed gene expression
        given unperturbed expression.

        Args:
            input_dim (int): Number of input features (genes).
            hidden_dim (int): Size of the hidden layer(s).
            output_dim (int): Number of output features (genes).
                             If None, defaults to the same as input_dim.
        """
        super(SimpleNN, self).__init__()
        if output_dim is None:
            output_dim = input_dim

        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
        )

    def forward(self, x):
        return self.net(x)


def train_simple_nn(dataset, epochs=5, batch_size=32, lr=1e-3, device="cpu"):
    """
    Example training loop for a simple MLP that maps unperturbed -> perturbed gene expression.

    Args:
        dataset (Dataset): An instance of GeneExpressionDataset.
        epochs (int): Number of training epochs.
        batch_size (int): Batch size for DataLoader.
        lr (float): Learning rate for optimizer.
        device (str): "cpu" or "cuda".
    """
    # Use the first sample to infer the gene dimension
    sample = dataset[0]
    input_dim = sample["features"].shape[0]
    output_dim = sample["labels"].shape[0]

    model = SimpleNN(input_dim=input_dim, output_dim=output_dim).to(device)

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    model.train()
    for epoch in range(epochs):
        total_loss = 0.0
        for batch in dataloader:
            # 'features' and 'labels' are [batch_size, num_genes] each
            features = batch["features"].to(device)
            labels = batch["labels"].to(device)

            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * features.size(0)

        avg_loss = total_loss / len(dataset)
        print(f"Epoch [{epoch+1}/{epochs}] - Loss: {avg_loss:.4f}")

    return model

In [6]:
# Create the dataset. 'normalize' off for debugging
# dataset = GeneExpressionDataset(
#     controls_file=controls,
#     perturbations_file=perturbations,
#     plate_column="det_plate",
#     normalize=False,    # Turn off normalization for now
#     n_rows=1000
# )

# Train the simple NN
device = "cuda" if torch.cuda.is_available() else "cpu"
model = train_simple_nn(dataset, epochs=5, batch_size=32, lr=1e-3, device=device)

Epoch [1/5] - Loss: 49.8775
Epoch [2/5] - Loss: 40.6629
Epoch [3/5] - Loss: 33.0737
Epoch [4/5] - Loss: 26.5846
Epoch [5/5] - Loss: 21.0784
